### Vorverarbeitung von Verhaltensdaten in Python

Dieses ``Jupyter Notebook`` kann für die Vorverarbeitung eurer Experimente zur visuellen Suche benutzen werden. Um die Daten korrekt einzulesen und als gemeinsamen ``data frame (df)`` wieder auf die Festplatte zu schreiben, ist eine einheitliche Bennenung der relevanten Spalten notwendig. Dafür baue ich auf das Tutorial zur Programmierung von Experimenten auf, d. h. eine Versuchspersonen ID wird in die Spalte ``sub_id`` geschrieben, Reaktionszeiten in die Spalte ``reaction_time``, Angaben ob eine korrekte Eingabe durch die Versuchsperson erfolgte wird in die Spalte ``accuracy`` geschrieben, ... 

Die notwendigen Pakete um die hier vorgestellte Funktion zu nutzen sind ``os``, ``glob`` und die in sowohl in ``Psychopy`` als auch ``Anaconda`` vorhandenen Pakete ``Pandas`` und ``Numpy``. Für die eigentliche Auswertung wird weiterhin das ``Afex`` Paket in ``R`` notwendig sein. Solltet ihr Vorverarbeitung und Auswertung in ``R`` durchführen wollen, rate ich dazu dem Tutorial von Peter Vavra auf unserem ``YouTube`` Kanal zu folgen (https://www.youtube.com/watch?v=Pu63OMUISRk).

### Typisches Vorgehen bei der Vorverarbeitung von Reaktionszeitdaten
Ziel der Vorverarbeitung ist es **nicht** die Daten so zu trimmen, dass in der anschließenden Analyse signifikante Ergebnisse produziert werden. Vielmehr soll das ``Signal zu Rauschen`` Verhältnis verbessert werden, indem Versuchsdurchgänge die durch externe Ablenkungen (e. g. Geräusche im Raum führen in einem Trial zu extrem hoher Reaktionszeit), motorische Ungenauigkeit, usw. verursacht wurden aus dem Datensatz ausgeschlossen werden. Deshalb ist es **zwingend** erforderlich die (1) Ausschlusskriterien komplett zu benennen und (b) deskriptive Angaben zur Menge ausgeschlossener Daten zu machen. Typischerweise geht man im Bereich visuelle Suchexperimente dabei wie folgt vor:
- Ausschluss aller Versuchsdurchgänge mit fehlerhafter Reaktion
- Ausschluss aller Versuchsdurchgänge größer als 2x (oder mehr) ``Standardabweichung`` vom individuellen arithmentischen Mittel
- Ausschluss aller Versuchsdurchgänge unter 200 ms
<br><br>
``Wichtig:`` Standardabweichungen sind pro Versuchsperson und nicht für die gesamte Gruppe zu berechnen!

In [1]:
import os
import glob
import numpy as np
import pandas as pd

### Einlesen der Pfade unabhängig vom jeweiligen Betriebssystem
Die folgende Funktion ist dafür gedacht Pfade zu den ``.tsv`` Dateien eurer Versuchspersonen einzulesen, unabhängig vom jeweiligen Betriebssystem. Da ein Großteil von euch ``Windows`` Systeme benutzt, sind die hier angegebenen Pfade auf einem ``Windows`` System entstanden. Das ``os`` Paket liefert aber bei korrekter Verwendung auch Pfade für ``Unix`` Systeme (``MacOS`` und ``Linux``). <br> 

Zur korrekten Durchführung muss das ``Jupyter Notebook`` im Ordner über dem Datenordner (hier ``\\data\\`` liegen. Bei Ausführen der Funktion kann aber auch ein anderes Verzeichnis als ``data_dir`` Argument übergeben werden.

In [2]:
def get_paths(data_dir="data"):
    path_list = glob.glob(os.path.join(os.getcwd(), data_dir, "sub*.tsv"))
    print(f"availabe data sheets:\t {path_list}")
    return path_list

Hier wird die eigentliche Funktion aufgerufen und unsere Pfade werden in die path_list Variable gespeichert. Die 3 hier ausgeführten Dateien entsprechen den Vorgaben aus dem Cats vs. Humans Experiment.

In [3]:
path_list = get_paths(data_dir="data")

availabe data sheets:	 ['C:\\Users\\nico\\Desktop\\teaching\\data\\sub_01_cat-human.tsv', 'C:\\Users\\nico\\Desktop\\teaching\\data\\sub_02_cat-human.tsv', 'C:\\Users\\nico\\Desktop\\teaching\\data\\sub_03_cat-human.tsv']


### Erstellung der Funktion zur Datenvorverarbeitung
Im nächsten Schritt wollen wir die Ausschlusskriterien definieren und pro ``data frame`` (also pro Versuchsperson) die Vorverarbeiung laufen lassen. Außerdem müssen wir uns noch ausgeben lassen wieviele Daten tatsächlich ausgeschlossen wurden. Dazu lesen wir die Daten mittels ``Pandas`` ein und benutzen das ``sep`` Argument ``\t`` für Tabulator. Hier benutzen wir außerdem das Paket Numpy um Mittelwert und Standardabweichung pro Versuchsperson zu berechnen.<br><br>
``Wichtig:``
- Reaktionszeitdaten für sub-1 und sub-2 sind ``fast identisch``, sub-3 hingegen war deutlich langsamer!

``Ground-Truth:`` 
- sub-1 hat drei ``Datenpunkte``, die mit den hier gewählten Parametern ausgeschlossen werden müssen und sub-2 hat zwei ``Datenpunkte``
- Obwohl sub-3 deutlich langsamer ist, wird auf Basis der ``indidividuellen`` Parameter kein Datenpunkt aus der ``reaction_time`` Spalte ausgeschlossen
- Alle 3 Versuchspersonen haben jeweils 2x fehlerhaft reagiert

In [4]:
def preprocess_df(path):
    tmp_df = pd.read_csv(path, sep="\t")
    mean_rt, std_rt = np.mean(tmp_df["reaction_time"]), np.std(tmp_df["reaction_time"])
    
    # Speichern der Anzahl von "Rohdaten"
    rt_before = len(tmp_df["reaction_time"])
    
    # Hier erfolgt der Ausschluss für falsche Reaktionen (accuracy == "no")
    tmp_df = tmp_df[tmp_df["accuracy"] != "no"]
    
    # Berechnung eines numerischen Werts für langsame trials: Mittelwert + 2x Standardabweichung
    upper_cut = mean_rt + 2 * std_rt
    
    # Ausgabe von deskriptiven Variablen
    print("Subject:", tmp_df["sub_id"][0], f"Mean: {mean_rt} Std: {std_rt} Cut-off (upper) {upper_cut}")
    
    # Hier erfolgt der eigentlich Ausschluss für langsame Reaktionszeiten
    tmp_df = tmp_df[tmp_df["reaction_time"] <= upper_cut]

    # Aufstellung des Kriteriums: Reaktionszeiten schneller als 200 ms (nur für sub-1 und sub-2)
    lower_cut = 0.2
    
    # Hier erfolgt der eigentlich Ausschluss für zu schnelle Reaktionszeiten
    tmp_df = tmp_df[tmp_df["reaction_time"] >= lower_cut]
    
    # Speichern der Anzahl von vorverarbeiteten Daten
    rt_after = len(tmp_df["reaction_time"])
    
    print(f"Anzahl an Rohdaten: {rt_before};\t Anzahl an Vorverarbeiteten Daten: {rt_after}")
    print("\t")
    return tmp_df

### Aufrufen der Vorverarbeitungsfunktion für alle Datensätze
Hier nutzen wir unsere ``path_list`` um jede Versuchsperson einzeln einzulesen und individuelle Cut-Off Werte zu ermitteln. Zusätzlich wollen wir eine ``.tsv`` mit allen (vorverarbeiteten) Daten erzeugen, um diese einfacher in ``R`` einlesen zu können. Dazu erzeugen wir einfach eine Liste, fügen dieser Liste die Daten mittels ``for loop`` hinzu und ``verknüpfen (concatenate)`` sie anschließend.

In [5]:
df_container = []

for path in path_list:
    df_container.append(preprocess_df(path))

df_all = pd.concat(df_container)
df_all

Subject: 1 Mean: 1.38125 Std: 0.759615651168405 Cut-off (upper) 2.90048130233681
Anzahl an Rohdaten: 40;	 Anzahl an Vorverarbeiteten Daten: 35
	
Subject: 2 Mean: 1.394 Std: 0.7281510832238046 Cut-off (upper) 2.850302166447609
Anzahl an Rohdaten: 40;	 Anzahl an Vorverarbeiteten Daten: 36
	
Subject: 3 Mean: 2.445 Std: 0.11543396380615198 Cut-off (upper) 2.6758679276123036
Anzahl an Rohdaten: 40;	 Anzahl an Vorverarbeiteten Daten: 38
	


,sub_id,age,sex,glasses,block,trial,reaction_time,accuracy
1,1,32,m,yes,1,2,1.25,yes
2,1,32,m,yes,1,3,1.25,yes
3,1,32,m,yes,1,4,1.25,yes
4,1,32,m,yes,1,5,1.25,yes
5,1,32,m,yes,1,6,1.25,yes
...,...,...,...,...,...,...,...,...
35,3,19,f,yes,2,36,2.60,yes
36,3,19,f,yes,2,37,2.61,yes
37,3,19,f,yes,2,38,2.62,yes
38,3,19,f,yes,2,39,2.63,yes


### Speichern
Der so entstandene ``df`` wird dann als ``complete_df.tsv`` in den Ordner geschrieben aus dem das ``Jupyter Notebook`` heraus ausgeführt wurde und kann in ``R`` eingelesen werden.

In [6]:
df_all.to_csv(os.path.join(os.getcwd(), "complete_df.tsv"), sep="\t", index=False)

### Hilfe bei Abweichungen von unserem vorgebenen Schema: Dateiformat
Da nicht alle Gruppen das von uns vorgeschlagene Format zum Erstellen der ``.tsv`` Dateien eingehalten haben, hier noch ein paar kurze Hilfestellungen für die gängigsten Probleme: 
- Datei liegt nicht als ``.tsv`` sondern als ``.csv`` Datei vor: ``"sub*.tsv"`` zu  ``"sub*.csv"`` ändern und ``pd.read_csv(path, sep="\t")`` zu ``pd.read_csv(path, sep=",")``

In [7]:
def get_paths(data_dir="data"):
    path_list = glob.glob(os.path.join(os.getcwd(), data_dir, "*.csv"))
    print(f"availabe data sheets:\t {path_list}")
    return path_list

In [8]:
path_list = get_paths(data_dir="data")

availabe data sheets:	 ['C:\\Users\\nico\\Desktop\\teaching\\data\\results_1.csv', 'C:\\Users\\nico\\Desktop\\teaching\\data\\results_2.csv', 'C:\\Users\\nico\\Desktop\\teaching\\data\\results_3.csv']


### Hilfe bei Abweichungen von unserem vorgebenen Schema: Fehlende sub_id Spalte
- Da wir eine ``.tsv`` Datei pro Versuchsperson erhalten und lediglich einen Indikator für den kompletten ``df`` brauchen, um die Daten korrekt zu trennen, können wir die Eintragung händisch vornehmen. Dazu erstellen wir eine Liste von Integern die so viele Listenelemente hat wie wir Versuchspersonen gemessen haben.

In [9]:
# Anzahl der Versuchspersonen == Anzahl der Dateien
# Wir addieren 1 hinzu, da Python von 0 anfängt zu zählen
sub_list = list(range(1, len(path_list) + 1))
sub_list

[1, 2, 3]

Hier finden Anpassungen auf Basis der Spaltenbenennung einer einzelnen Gruppe statt (e.g. ``reaction_time`` wird zu ``Reaktionszeit``.

In [10]:
# Wir ergänzen das sub_id Argument in unserer Vorverarbeitungsfunktion 
def preprocess_df(path, sub_id):
    tmp_df = pd.read_csv(path, sep=",")
    mean_rt, std_rt = np.mean(tmp_df["Reaktionszeit"]), np.std(tmp_df["Reaktionszeit"])
    
    # Einfügen einer Spalte für die Versuchspersonen ID
    # Wiederholungen so oft wie es Reaktionszeit Zellen gibt
    tmp_df["sub_id"] = np.repeat(sub_id, len(tmp_df["Reaktionszeit"]))

    # Speichern der Anzahl von "Rohdaten"
    rt_before = len(tmp_df["Reaktionszeit"])
    
    # Berechnung eines numerischen Werts für langsame trials: Mittelwert + 2x Standardabweichung
    upper_cut = mean_rt + 3 * std_rt
    
    # Ausgabe von deskriptiven Variablen
    print("Subject:", sub_id, f"Mean: {mean_rt} Std: {std_rt} Cut-off (upper) {upper_cut}")
    
    # Hier erfolgt der eigentlich Ausschluss für langsame Reaktionszeiten
    tmp_df = tmp_df[tmp_df["Reaktionszeit"] <= upper_cut]

    # Aufstellung des Kriteriums: Reaktionszeiten schneller als 200 ms (nur für sub-1 und sub-2)
    lower_cut = 0.2
    
    # Hier erfolgt der eigentlich Ausschluss für zu schnelle Reaktionszeiten
    tmp_df = tmp_df[tmp_df["Reaktionszeit"] >= lower_cut]
    
    # Speichern der Anzahl von Reaktionszeit Daten
    rt_after = len(tmp_df["Reaktionszeit"])
    
    print(f"Anzahl an Rohdaten: {rt_before};\t Anzahl an Vorverarbeiteten Daten: {rt_after}")
    print("\t")
    return tmp_df

Wir führen eine neue Variable ein, die mit jeder Iteration des Loops durchzählt (sog. ``Inkrementieren``) und und so eine neue sub_id pro ``df`` anlegt. Das Konzept eines ``Inkrements`` sollte euch noch aus den ``Python Basics II`` Video bekannt sein (für weitere Erläuterungen: https://www.youtube.com/watch?v=YHGavaIjpm).

In [11]:
df_container = []
inkrement = 0

for path in path_list:
    df_container.append(preprocess_df(path, sub_id=sub_list[inkrement]))
    inkrement += 1

df_all = pd.concat(df_container)
df_all

Subject: 1 Mean: 1.1807725694444444 Std: 0.7212000925487714 Cut-off (upper) 3.3443728470907583
Anzahl an Rohdaten: 360;	 Anzahl an Vorverarbeiteten Daten: 356
	
Subject: 2 Mean: 1.7197916666666666 Std: 1.6558381137481322 Cut-off (upper) 6.687306007911063
Anzahl an Rohdaten: 360;	 Anzahl an Vorverarbeiteten Daten: 354
	
Subject: 3 Mean: 1.5521701388888889 Std: 0.8790421019835756 Cut-off (upper) 4.1892964448396155
Anzahl an Rohdaten: 360;	 Anzahl an Vorverarbeiteten Daten: 358
	


,Block,Bildnummer,Perspektive,Zustand,Antwort,Antwort ausgewertet,Reaktionszeit,Geschlecht,Alter,sub_id
0,1.0,3221,P1,S2,y,Richtig,0.906250,m,22,1
1,1.0,3222,P2,S2,n,Falsch,1.109375,m,22,1
2,1.0,1211,P1,S1,y,Richtig,1.109375,m,22,1
3,1.0,1212,P2,S1,y,Richtig,0.734375,m,22,1
4,1.0,1321,P1,S2,y,Richtig,1.843750,m,22,1
...,...,...,...,...,...,...,...,...,...,...
355,6.0,1422,P2,S2,y,Richtig,1.500000,m,23,3
356,6.0,2211,P1,S1,y,Richtig,1.093750,m,23,3
357,6.0,2212,P2,S1,y,Richtig,1.296875,m,23,3
358,6.0,2931,P1,S2,y,Richtig,1.046875,m,23,3


In [12]:
df_all.to_csv(os.path.join(os.getcwd(), "complete_df_julian.tsv"), sep="\t", index=False)